In [1]:
import pandas as pd
import os
import lightgbm as lgb
import xgboost as xgb
import numpy as np
import matplotlib.pyplot as plt
import time

from math import sqrt
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, cross_val_score
from sklearn.linear_model import Lasso
from sklearn.ensemble import GradientBoostingRegressor

import keras
from keras.layers.normalization import BatchNormalization
from keras.models import Sequential
from keras.layers import Dense, Activation, LSTM
from keras.callbacks import ModelCheckpoint
import pickle

Using TensorFlow backend.


In [2]:
price = pd.read_csv('./transpose-123 - transpose.csv')
path = 'pepper_supply/pepper'
o00 = pd.read_excel('./{}_00.xlsx'.format(path))
o01 = pd.read_excel('./{}_01.xlsx'.format(path))
o02 = pd.read_excel('./{}_02.xlsx'.format(path))
o03 = pd.read_excel('./{}_03.xlsx'.format(path))
o04 = pd.read_excel('./{}_04.xlsx'.format(path))
o05 = pd.read_excel('./{}_05.xlsx'.format(path))
o06 = pd.read_excel('./{}_06.xlsx'.format(path))
o07 = pd.read_excel('./{}_07.xlsx'.format(path))
o08 = pd.read_excel('./{}_08.xlsx'.format(path))
o09 = pd.read_excel('./{}_09.xlsx'.format(path))
o10 = pd.read_excel('./{}_10.xlsx'.format(path))
o11 = pd.read_excel('./{}_11.xlsx'.format(path))
o12 = pd.read_excel('./{}_12.xlsx'.format(path))
o13 = pd.read_excel('./{}_13.xlsx'.format(path))
o14 = pd.read_excel('./{}_14.xlsx'.format(path))
o15 = pd.read_excel('./{}_15.xlsx'.format(path))
o16 = pd.read_excel('./{}_16.xlsx'.format(path))
o17 = pd.read_excel('./{}_17.xlsx'.format(path))
o18 = pd.read_excel('./{}_18.xlsx'.format(path))
o19 = pd.read_excel('./{}_19.xlsx'.format(path))
# o20 = pd.read_excel('./{}_20.xlsx'.format(path))

supply = pd.concat([o00, o01, o02, o03, o04, o05, o06, o07, o08, o09, o10, o11, o12, o13, o14, o15, o16, o17, o18, o19])
# supply = supply[2:]
supply['년도'] = supply['거래년월일'].apply(lambda x: str(x)[:4]).astype(str)
supply['월'] = supply['거래년월일'].apply(lambda x: str(x)[4:6]).astype(str)
supply['일'] = supply['거래년월일'].apply(lambda x: str(x)[6:8]).astype(str)
supply['거래량'] = supply['거래량'].apply(lambda x: ''.join(str(x).split(','))).astype(int)
sup_total = supply.groupby(['년도','월','일'], as_index=False).sum()
sup_total = sup_total[:-2]
sup_total.head()

,거래년월일,품목명,시장코드,시장명,출하지코드,출하지명,거래량,품목코드,년도,월,일
0,총 합계,전체,NaN,NaN,NaN,NaN,79211,NaN,총 합계,,
1,소계,건고추(일반),NaN,NaN,NaN,NaN,79211,120701.0,소계,,
2,20001230,건고추(일반),210001.0,부산엄궁도매시장,356010,충청남도서산시동문동,5100,120701.0,2000,12,30
3,20001229,건고추(일반),311201.0,구리도매시장,501000,광주동구,60,120701.0,2000,12,29
4,20001229,건고추(일반),210001.0,부산엄궁도매시장,621010,경상남도김해시부원동,4500,120701.0,2000,12,29


In [3]:
price['년도'] = price['년도'].apply(lambda x: str(x))
price['월'] = price['월/일'].apply(lambda x: str(x)[0:2]).astype(str)
price['일'] = price['월/일'].apply(lambda x: str(x)[3:5]).astype(str)
price.head()

,년도,월/일,평균,월,일
0,2000,01/03,"103,000",01,03
1,2000,01/04,"97,000",01,04
2,2000,01/05,"95,000",01,05
3,2000,01/06,"98,000",01,06
4,2000,01/07,"98,000",01,07


In [5]:
df = pd.merge(price, sup_total, on=['년도','월','일'])
df.head()

,년도,월/일,평균,월,일,시장코드,거래량,품목코드
0,2000,01/03,"103,000",01,03,380201.0,130,120701.0
1,2000,01/06,"98,000",01,06,380201.0,130,120701.0
2,2000,01/12,"97,000",01,12,380201.0,130,120701.0
3,2000,01/14,"95,000",01,14,380201.0,130,120701.0
4,2000,01/15,"91,000",01,15,380201.0,130,120701.0


In [6]:
df2 = df[['년도','월','일','평균','거래량']]
df2['년도'] = df2['년도'].apply(lambda x: int(x))
df2['월'] = df2['월'].apply(lambda x: int(x))
df2['일'] = df2['일'].apply(lambda x: int(x))
df2['평균'] = df2['평균'].apply(lambda x: ''.join(str(x).split(','))).astype(int) 

df2

/home/florence/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/florence/.local/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/florence/.local/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead


,년도,월,일,평균,거래량
0,2000,1,3,103000,130
1,2000,1,6,98000,130
2,2000,1,12,97000,130
3,2000,1,14,95000,130
4,2000,1,15,91000,130
5,2000,1,25,95000,130
6,2000,1,27,95000,130
7,2000,1,28,95000,130
8,2000,1,31,95000,130
9,2000,2,8,90000,489


In [7]:
a = list(df2['평균'])
df2['d+1'] = a
a.insert(0,0)
a = a[:-1]

df2['d-0'] = a
a.insert(0,0)
a = a[:-1]

df2['d-1'] = a
a.insert(0,0)
a = a[:-1]

df2['d-2'] = a
a.insert(0,0)
a = a[:-1]

df2['d-3'] = a
a.insert(0,0)
a = a[:-1]

df2['d-4'] = a
a.insert(0,0)
a = a[:-1]

df2['d-5'] = a
a.insert(0,0)
a = a[:-1]

df2['d-6'] = a
a.insert(0,0)
a = a[:-1]

df2['d-7'] = a
df2.head(10)

/home/florence/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/florence/.local/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,년도,월,일,평균,거래량,d+1,d-0,d-1,d-2,d-3,d-4,d-5,d-6,d-7
0,2000,1,3,103000,130,103000,0,0,0,0,0,0,0,0
1,2000,1,6,98000,130,98000,103000,0,0,0,0,0,0,0
2,2000,1,12,97000,130,97000,98000,103000,0,0,0,0,0,0
3,2000,1,14,95000,130,95000,97000,98000,103000,0,0,0,0,0
4,2000,1,15,91000,130,91000,95000,97000,98000,103000,0,0,0,0
5,2000,1,25,95000,130,95000,91000,95000,97000,98000,103000,0,0,0
6,2000,1,27,95000,130,95000,95000,91000,95000,97000,98000,103000,0,0
7,2000,1,28,95000,130,95000,95000,95000,91000,95000,97000,98000,103000,0
8,2000,1,31,95000,130,95000,95000,95000,95000,91000,95000,97000,98000,103000
9,2000,2,8,90000,489,90000,95000,95000,95000,95000,91000,95000,97000,98000


In [8]:
del df2['평균']
df2 = df2[8:]
df2 = df2.reset_index(drop=True)
df2.head(10)

,년도,월,일,거래량,d+1,d-0,d-1,d-2,d-3,d-4,d-5,d-6,d-7
0,2000,1,31,130,95000,95000,95000,95000,91000,95000,97000,98000,103000
1,2000,2,8,489,90000,95000,95000,95000,95000,91000,95000,97000,98000
2,2000,2,12,130,64000,90000,95000,95000,95000,95000,91000,95000,97000
3,2000,2,14,130,57000,64000,90000,95000,95000,95000,95000,91000,95000
4,2000,2,19,720,40000,57000,64000,90000,95000,95000,95000,95000,91000
5,2000,2,28,4900,38000,40000,57000,64000,90000,95000,95000,95000,95000
6,2000,3,6,78,39400,38000,40000,57000,64000,90000,95000,95000,95000
7,2000,3,7,608,41400,39400,38000,40000,57000,64000,90000,95000,95000
8,2000,3,10,69,48800,41400,39400,38000,40000,57000,64000,90000,95000
9,2000,3,14,24,49200,48800,41400,39400,38000,40000,57000,64000,90000


In [9]:
y = df2['d+1']
del df2['d+1']

In [10]:

model = Sequential()

model.add(Dense(30, input_dim=12))
model.add(BatchNormalization())
model.add(Activation('relu'))          
model.add(Dense(900))
model.add(Activation('relu'))
model.add(Dense(4096))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dense(1028))
model.add(Activation('relu'))
model.add(Dense(512))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dense(128))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dense(32))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dense(16))
model.add(Activation('relu'))
model.add(Dense(8))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dense(4))
model.add(Activation('relu'))

model.add(Dense(1, activation='relu'))
model.compile(loss='mean_squared_error',
             optimizer='adam',
             metrics=['accuracy'])


seed_pred_result = []
rmse_result = []
for i in range(10):
    rand_num = i
    x_train, x_test,y_train, y_test = train_test_split(df2, y, test_size=0.3, random_state=rand_num)

    gboost = GradientBoostingRegressor(random_state=rand_num)
    xgboost = xgb.XGBRegressor(random_state=rand_num)
    lightgbm = lgb.LGBMRegressor(random_state=rand_num)
    lasso = Lasso(alpha=1.0, random_state=rand_num)

    models = [{'model':xgboost, 'name':'XGBoost'},
              {'model':gboost, 'name': 'GradientBoosting'},
              {'model':lightgbm, 'name':'LightGBM'},
              {'model':lasso, 'name': 'Lasso'}]
    result = []
    rmse = []
    result.append(y_test)

    for m in models:
        m['model'].fit(x_train.values, y_train)
        pickle.dump(m['model'], open("pima_{}_{}.pickle.dat".format(i, m['name']), "wb"))
        #.save_model('./{}_model'.format(m['name']))

    for m in models:
    #         pickle.dump(m['model'], open("pima_{}.pickle.dat".format(m['name']), "wb"))
        loaded_model = pickle.load(open("pima_{}_{}.pickle.dat".format(i, m['name']), "rb"))
    #     loaded_model.predict(X_test)
        predictions = loaded_model.predict(x_test.values) 
        result.append(predictions)
        rmse.append({m['name']:sqrt(mean_squared_error(y_test, predictions))})

#     x_train = keras.preprocessing.sequence.pad_sequences(x_train, maxlen=80)
#     x_test = keras.preprocessing.sequence.pad_sequences(x_test, maxlen=80)
    
    # 모델 저장 폴더 설정
    MODEL_DIR = './model_{}/'.format(i)
    if not os.path.exists(MODEL_DIR):
        os.mkdir(MODEL_DIR)
        
    # 모델 저장 조건 설정
    modelpath = MODEL_DIR + 'keras_{epoch:02d}-{val_loss:.4f}.hdf5'
    checkpointer = ModelCheckpoint(filepath=modelpath, monitor='val_loss', verbose=1, save_best_only=True)
    early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=130)
    def scheduler(epoch):
        if epoch > 80:
            return 0.004
        if epoch < 170:
            return 0.003
        if epoch < 260:
            return 0.002
        if epoch < 350:
            return 0.001
        else:
            return 0.005
        
    lr_schedule = keras.callbacks.LearningRateScheduler(scheduler)
    
    history = model.fit(x_train, y_train, validation_split=0.3, epochs=500, batch_size=32, callbacks=[checkpointer, early_stop, lr_schedule])
    
    
    modelPred = model.predict(x_test)
    Y_prediction = modelPred.flatten()

    result.append(Y_prediction)
    rmse.append({'keras':sqrt(mean_squared_error(y_test, modelPred))})

    dict = {'y': result[0], 'xgboost': result[1], 'gboost': result[2], 'lightgbm':result[3], 'lasso':result[4],'keras': result[5]}

    df_res = pd.DataFrame(dict)
    df_res['average'] = df_res[['xgboost', 'gboost', 'lightgbm', 'lasso', 'keras']].apply(lambda x: round(x.mean()), axis=1)
    rmse.append({'average':sqrt(mean_squared_error(y_test, df_res['average']))})

    rmse_result.append(rmse)
    seed_pred_result.append(df_res)    

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


/home/florence/.local/lib/python3.6/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 31528963418.922848, tolerance: 342007222.08415496
  positive)


Train on 2058 samples, validate on 882 samples
Epoch 1/500
2058/2058 [==============================] - 3s 1ms/step - loss: 4756304948.7425 - acc: 0.0000e+00 - val_loss: 4538011736.2358 - val_acc: 0.0000e+00

Epoch 00001: val_loss improved from inf to 4538011736.23583, saving model to ./model_0/keras_01-4538011736.2358.hdf5
Epoch 2/500
2058/2058 [==============================] - 1s 297us/step - loss: 4755925794.0836 - acc: 0.0000e+00 - val_loss: 4537176851.1565 - val_acc: 0.0000e+00

Epoch 00002: val_loss improved from 4538011736.23583 to 4537176851.15646, saving model to ./model_0/keras_02-4537176851.1565.hdf5
Epoch 3/500
2058/2058 [==============================] - 1s 296us/step - loss: 4755006375.9300 - acc: 0.0000e+00 - val_loss: 4532904806.7483 - val_acc: 0.0000e+00

Epoch 00003: val_loss improved from 4537176851.15646 to 4532904806.74830, saving model to ./model_0/keras_03-4532904806.7483.hdf5
Epoch 4/500
2058/2058 [==============================] - 1s 298us/step - loss: 4753163

2058/2058 [==============================] - 1s 291us/step - loss: 3226623778.9543 - acc: 0.0000e+00 - val_loss: 2301115250.3583 - val_acc: 0.0000e+00

Epoch 00030: val_loss did not improve from 2270785033.28798
Epoch 31/500
2058/2058 [==============================] - 1s 311us/step - loss: 3107391886.3052 - acc: 0.0000e+00 - val_loss: 2711900070.6032 - val_acc: 0.0000e+00

Epoch 00031: val_loss did not improve from 2270785033.28798
Epoch 32/500
2058/2058 [==============================] - 1s 365us/step - loss: 2993483887.4558 - acc: 0.0000e+00 - val_loss: 2369264176.7619 - val_acc: 0.0000e+00

Epoch 00032: val_loss did not improve from 2270785033.28798
Epoch 33/500
2058/2058 [==============================] - 1s 389us/step - loss: 2867676513.1506 - acc: 0.0000e+00 - val_loss: 2911293629.2426 - val_acc: 0.0000e+00

Epoch 00033: val_loss did not improve from 2270785033.28798
Epoch 34/500
2058/2058 [==============================] - 1s 323us/step - loss: 2751592325.5977 - acc: 0.0000e+00

2058/2058 [==============================] - 1s 353us/step - loss: 242883645.2012 - acc: 0.0000e+00 - val_loss: 231683277.2063 - val_acc: 0.0000e+00

Epoch 00065: val_loss did not improve from 162523840.07256
Epoch 66/500
2058/2058 [==============================] - 1s 350us/step - loss: 210390506.0914 - acc: 4.8591e-04 - val_loss: 205125567.1293 - val_acc: 0.0000e+00

Epoch 00066: val_loss did not improve from 162523840.07256
Epoch 67/500
2058/2058 [==============================] - 1s 327us/step - loss: 231528412.0350 - acc: 0.0000e+00 - val_loss: 242084213.1156 - val_acc: 0.0000e+00

Epoch 00067: val_loss did not improve from 162523840.07256
Epoch 68/500
2058/2058 [==============================] - 1s 348us/step - loss: 187245303.1215 - acc: 0.0000e+00 - val_loss: 392195227.5737 - val_acc: 0.0000e+00

Epoch 00068: val_loss did not improve from 162523840.07256
Epoch 69/500
2058/2058 [==============================] - 1s 431us/step - loss: 172158727.1059 - acc: 0.0000e+00 - val_loss: 

...

2058/2058 [==============================] - 1s 314us/step - loss: 63747598.0330 - acc: 0.0000e+00 - val_loss: 350627327816.1995 - val_acc: 0.0000e+00

Epoch 00111: val_loss did not improve from 347896934.01361
Epoch 112/500
2058/2058 [==============================] - 1s 318us/step - loss: 64526610.6589 - acc: 4.8591e-04 - val_loss: 810827214058.3765 - val_acc: 0.0000e+00

Epoch 00112: val_loss did not improve from 347896934.01361
Epoch 113/500
2058/2058 [==============================] - 1s 350us/step - loss: 54814318.2119 - acc: 0.0000e+00 - val_loss: 233869759810.6848 - val_acc: 0.0000e+00

Epoch 00113: val_loss did not improve from 347896934.01361
Epoch 114/500
2058/2058 [==============================] - 1s 393us/step - loss: 56854725.6385 - acc: 0.0000e+00 - val_loss: 175689972387.4830 - val_acc: 0.0000e+00

Epoch 00114: val_loss did not improve from 347896934.01361
Epoch 115/500
2058/2058 [==============================] - 1s 396us/step - loss: 71770949.8931 - acc: 0.0000e+00 -

2058/2058 [==============================] - 1s 334us/step - loss: 65425503.9689 - acc: 0.0000e+00 - val_loss: 13451528895.1474 - val_acc: 0.0000e+00

Epoch 00147: val_loss did not improve from 85280876.73469
Epoch 148/500
2058/2058 [==============================] - 1s 356us/step - loss: 82193019.3819 - acc: 0.0000e+00 - val_loss: 30734584612.3900 - val_acc: 0.0000e+00

Epoch 00148: val_loss did not improve from 85280876.73469
Epoch 149/500
2058/2058 [==============================] - 1s 407us/step - loss: 66432015.6735 - acc: 4.8591e-04 - val_loss: 47204455486.1315 - val_acc: 0.0000e+00

Epoch 00149: val_loss did not improve from 85280876.73469
Epoch 150/500
2058/2058 [==============================] - 1s 410us/step - loss: 51532079.9864 - acc: 0.0000e+00 - val_loss: 20818794897.9229 - val_acc: 0.0000e+00

Epoch 00150: val_loss did not improve from 85280876.73469
Epoch 151/500
2058/2058 [==============================] - 1s 295us/step - loss: 51085098.1069 - acc: 0.0000e+00 - val_los


Epoch 00183: val_loss did not improve from 85280876.73469
Epoch 184/500
2058/2058 [==============================] - 1s 323us/step - loss: 51358319.3314 - acc: 4.8591e-04 - val_loss: 452461266502.8934 - val_acc: 0.0000e+00

Epoch 00184: val_loss did not improve from 85280876.73469
Epoch 185/500
2058/2058 [==============================] - 1s 339us/step - loss: 62785265.2439 - acc: 0.0000e+00 - val_loss: 433759116091.0657 - val_acc: 0.0000e+00

Epoch 00185: val_loss did not improve from 85280876.73469
Epoch 186/500
2058/2058 [==============================] - 1s 478us/step - loss: 62699749.5821 - acc: 0.0000e+00 - val_loss: 571379251383.9275 - val_acc: 0.0000e+00

Epoch 00186: val_loss did not improve from 85280876.73469
Epoch 187/500
2058/2058 [==============================] - 1s 389us/step - loss: 68922586.5578 - acc: 0.0000e+00 - val_loss: 380553789043.5465 - val_acc: 0.0000e+00

Epoch 00187: val_loss did not improve from 85280876.73469
Epoch 188/500
2058/2058 [====================

2058/2058 [==============================] - 1s 340us/step - loss: 54877471.2789 - acc: 0.0000e+00 - val_loss: 743876267946.5669 - val_acc: 0.0011

Epoch 00220: val_loss did not improve from 85280876.73469
Epoch 221/500
2058/2058 [==============================] - 1s 312us/step - loss: 54349489.6327 - acc: 0.0000e+00 - val_loss: 866648594036.5896 - val_acc: 0.0000e+00

Epoch 00221: val_loss did not improve from 85280876.73469
Epoch 222/500
2058/2058 [==============================] - 1s 324us/step - loss: 66862712.8241 - acc: 0.0000e+00 - val_loss: 642596094010.1224 - val_acc: 0.0023

Epoch 00222: val_loss did not improve from 85280876.73469
Epoch 223/500
2058/2058 [==============================] - 1s 413us/step - loss: 55057727.3392 - acc: 0.0000e+00 - val_loss: 594867595237.6689 - val_acc: 0.0000e+00

Epoch 00223: val_loss did not improve from 85280876.73469
Epoch 224/500
2058/2058 [==============================] - 1s 377us/step - loss: 63628911.3703 - acc: 0.0000e+00 - val_loss: 4


Epoch 00256: val_loss did not improve from 85280876.73469
Epoch 257/500
2058/2058 [==============================] - 1s 333us/step - loss: 47891479.0515 - acc: 0.0000e+00 - val_loss: 145980867735.3107 - val_acc: 0.0000e+00

Epoch 00257: val_loss did not improve from 85280876.73469
Epoch 258/500
2058/2058 [==============================] - 1s 360us/step - loss: 66618793.6249 - acc: 0.0000e+00 - val_loss: 113394211510.9297 - val_acc: 0.0000e+00

Epoch 00258: val_loss did not improve from 85280876.73469
Epoch 259/500
2058/2058 [==============================] - 1s 376us/step - loss: 62913843.4052 - acc: 0.0000e+00 - val_loss: 22540789058.5760 - val_acc: 0.0000e+00

Epoch 00259: val_loss did not improve from 85280876.73469
Epoch 260/500
2058/2058 [==============================] - 1s 365us/step - loss: 46074538.8688 - acc: 4.8591e-04 - val_loss: 14565933188.5533 - val_acc: 0.0000e+00

Epoch 00260: val_loss did not improve from 85280876.73469
Epoch 261/500
2058/2058 [======================

In [11]:
# 다른 주소에서 다른 화면을 보여주는것을 라우팅이라 함


In [12]:
rmse_result[:4]

[[{'XGBoost': 5060.752739311941},
  {'GradientBoosting': 4679.093068874934},
  {'LightGBM': 4694.788908055785},
  {'Lasso': 4574.050564366571},
  {'keras': 11313.86342722908},
  {'average': 4846.1116553429465}],
 [{'XGBoost': 5416.473316056296},
  {'GradientBoosting': 5170.238447805436},
  {'LightGBM': 5117.617657159788},
  {'Lasso': 5013.438734620082},
  {'keras': 10216.28112896754},
  {'average': 5296.416434045113}],
 [{'XGBoost': 5034.9197538315675},
  {'GradientBoosting': 4908.920440277898},
  {'LightGBM': 4875.468595365591},
  {'Lasso': 4797.315143234106},
  {'keras': 11291.59811452594},
  {'average': 5109.3427015799925}],
 [{'XGBoost': 5254.526983305061},
  {'GradientBoosting': 5113.074539371874},
  {'LightGBM': 5014.7846134739475},
  {'Lasso': 4955.744830665329},
  {'keras': 7532.4743448721565},
  {'average': 4983.58743445837}]]

In [13]:
seed_pred_result[0].head(5)

,y,xgboost,gboost,lightgbm,lasso,keras,average
3037,78000,77643.085938,79549.740315,78961.813122,79688.783591,78837.929688,78936.0
3485,116000,97584.453125,109260.800027,111006.961995,106215.009763,94243.156250,103662.0
3347,78000,77754.031250,76693.638870,75745.685999,76025.363663,76447.554688,76533.0
72,34400,33556.945312,32449.101886,33156.258358,34357.203034,44545.757812,35613.0
2284,39000,43049.339844,40325.377431,39325.896702,39951.073014,50665.855469,42664.0


In [14]:
seed_pred_result[1].head(5)

,y,xgboost,gboost,lightgbm,lasso,keras,average
3604,51400,53590.417969,56774.975719,56236.419550,53327.508063,51106.898438,54207.0
781,53200,57254.070312,57399.782426,57329.140359,57031.358997,56312.933594,57065.0
2165,161000,157822.250000,156217.165358,155100.965332,153975.142609,128831.125000,150389.0
1217,101600,104559.054688,101591.077046,105369.990996,103347.849189,89657.968750,100905.0
1006,72400,69703.015625,69385.737561,71684.419765,69058.601280,64844.773438,68935.0


In [15]:
seed_pred_result[2].head(5)

,y,xgboost,gboost,lightgbm,lasso,keras,average
1296,37800,36095.246094,34892.166593,35363.262300,33998.052972,42034.410156,36477.0
140,74600,72049.625000,71173.107515,71495.336021,70407.348515,67830.531250,70591.0
2560,74400,73620.875000,75019.210238,75308.792053,74541.861075,71080.335938,73914.0
674,22800,22475.433594,22891.447140,22674.603345,22683.428196,31146.013672,24374.0
118,57000,52919.070312,56510.145409,56281.963678,55206.428533,57182.250000,55620.0


In [16]:
seed_pred_result[3].head(5)

,y,xgboost,gboost,lightgbm,lasso,keras,average
3038,78000,77255.460938,75005.761791,74504.676684,76436.848256,79162.000000,76473.0
953,45600,44912.820312,46070.239692,46094.166283,45075.216069,51509.792969,46732.0
3532,105600,102964.546875,102957.273431,101921.459843,101772.846410,99295.375000,101782.0
725,33600,34180.917969,34387.353145,34732.350896,34778.670181,41756.000000,35967.0
3681,47600,44954.175781,45314.531320,44959.526504,46490.130764,52370.289062,46818.0


In [17]:
seed_pred_result[4].head(5)

,y,xgboost,gboost,lightgbm,lasso,keras,average
1286,27600,28654.410156,27974.018698,28069.452178,26756.535605,33014.648438,28894.0
1456,24000,23229.611328,23697.086600,23321.374052,23746.970950,29771.416016,24753.0
2025,77800,75348.648438,76181.053237,74252.195973,75613.263250,82030.296875,76685.0
1287,31800,26375.537109,28245.530188,28172.273770,27546.587748,33295.375000,28727.0
3888,63000,65322.820312,66940.415930,67721.033368,66037.599129,72565.328125,67717.0


In [18]:
# model = Sequential()
# model.add(LSTM(50, activation='relu', input_shape=(7,3)))
# model.add(Dense(1))
# model.compile(optimizer='adam',loss='mse',metrics=['accuracy'])
# model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 50)                10800     
_________________________________________________________________
dense_14 (Dense)             (None, 1)                 51        
Total params: 10,851
Trainable params: 10,851
Non-trainable params: 0
_________________________________________________________________
